In [ ]:
#split (final): Filter out mononuclear copper protein
import os
import pandas as pd
import requests
import shutil
from collections import defaultdict
from Bio.PDB import PDBParser
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from Bio import SeqIO, pairwise2
import numpy as np
import time
import warnings

# Function to download PDB files
def download_pdb(pdb_code, save_directory):
    download_url = f"https://files.rcsb.org/download/{pdb_code}.pdb"
    response = requests.get(download_url)
    if response.status_code == 200:
        save_path = os.path.join(save_directory, f"{pdb_code}.pdb")
        with open(save_path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded {pdb_code}.pdb")
        return True
    else:
        print(f"Failed to download {pdb_code}.pdb")
        return False

# Function to categorize metal atoms in a PDB file
def categorize_metal_atoms(filename):
    pdb_id = os.path.basename(filename).split('.')[0]
    cu_atoms, other_metals = [], []

    with open(filename, 'r') as f:
        for line in f:
            if line.startswith('HETATM'):
                element_symbol = line[76:78].strip()
                if element_symbol == 'CU':
                    cu_atoms.append((pdb_id, 'CU'))
                elif element_symbol in ['ZN', 'MG', 'CO', 'CA', 'FE', 'PT', 'NA', 'K', 'LI', 'CD', 'YB', 'NI', 'PR', 'HG', 'MN']:
                    other_metals.append((pdb_id, element_symbol))

    return cu_atoms, other_metals

# Function to save metal atoms information to Excel and copy PDB files
def save_metal_atoms_to_excel(directory, destination):
    pdb_files = [f for f in os.listdir(directory) if f.endswith('.pdb')]
    all_cu_atoms, all_other_metals = [], []

    for pdb_file in pdb_files:
        cu_atoms, other_metals = categorize_metal_atoms(os.path.join(directory, pdb_file))
        all_cu_atoms.extend(cu_atoms)
        all_other_metals.extend(other_metals)

    pd.DataFrame(all_cu_atoms, columns=['PDB ID', 'Name of metal atoms']).to_excel(os.path.join(destination, 'cu_atoms.xlsx'), index=False)
    pd.DataFrame(all_other_metals, columns=['PDB ID', 'Name of metal atoms']).to_excel(os.path.join(destination, 'other_metals.xlsx'), index=False)

    for pdb_id in pd.read_excel(os.path.join(destination, 'cu_atoms.xlsx'))['PDB ID'].tolist():
        shutil.copy(os.path.join(directory, f"{pdb_id}.pdb"), destination)

    print("Metal atoms categorized and saved to Excel.")

# Function to count and categorize copper atoms in PDB files
def count_and_categorize_cu_atoms(excel_file, pdb_directory, output_excel):
    df = pd.read_excel(excel_file)
    cu_counts = df[df['Name of metal atoms'] == 'CU'].groupby('PDB ID').size().to_dict()

    result_df = pd.DataFrame(list(cu_counts.items()), columns=['PDB ID', 'Metal Count'])
    result_df.to_excel(output_excel, index=False)
    print(f"Metal atom counts have been saved to '{output_excel}'.")

    for pdb_id, count in cu_counts.items():
        src_file = os.path.join(pdb_directory, f"{pdb_id}.pdb")
        if os.path.isfile(src_file):
            dest_dir = os.path.join(pdb_directory, 'metal_count_1' if count == 1 else 'metal_count_greater_than_1')
            os.makedirs(dest_dir, exist_ok=True)
            shutil.copy(src_file, dest_dir)

    print("PDB files categorized and copied based on metal count.")

# Function to calculate the distance between two atoms
def calculate_distance(atom1, atom2):
    return np.linalg.norm(atom1.get_coord() - atom2.get_coord())

# Function to categorize PDB files based on copper and other metals within a distance threshold
def categorize_pdb_files(directory, distance_threshold=5.0):
    copper_ion, other_metals = 'CU', ['ZN', 'MG', 'CO', 'CA', 'FE', 'PT', 'NA', 'K', 'LI', 'CD', 'YB', 'NI', 'PR', 'HG', 'MN']
    mono_hetero_results, mono_results = [], []

    for pdb_file in (f for f in os.listdir(directory) if f.endswith('.pdb')):
        pdb_id = pdb_file.split('.')[0]
        structure = PDBParser().get_structure(pdb_id, os.path.join(directory, pdb_file))
        copper_atoms = [res for model in structure for chain in model for res in chain if res.get_resname() == copper_ion]

        if len(copper_atoms) == 1:
            copper_atom = copper_atoms[0]['CU']
            other_metals_near_copper = any(
                any(calculate_distance(copper_atom, atom) <= distance_threshold for atom in res if atom.element in other_metals)
                for model in structure for chain in model for res in chain
            )
            (mono_hetero_results if other_metals_near_copper else mono_results).append(pdb_id)

    save_results(mono_hetero_results, mono_results, directory)

# Function to categorize PDB files based on the presence of multiple copper atoms and other metals within a distance threshold
def categorize_pdb_files_multinuclear(directory, distance_threshold=5.0):
    copper_ion, other_metals = 'CU', ['ZN', 'MG', 'CO', 'CA', 'FE', 'PT', 'NA', 'K', 'LI', 'CD', 'YB', 'NI', 'PR', 'HG', 'MN']
    multi_hetero_results, multi_mono_results, homo_multinuclear_results = [], [], []

    for pdb_file in (f for f in os.listdir(directory) if f.endswith('.pdb')):
        pdb_id = pdb_file.split('.')[0]
        structure = PDBParser().get_structure(pdb_id, os.path.join(directory, pdb_file))
        copper_atoms = [res['CU'] for model in structure for chain in model for res in chain if res.get_resname() == copper_ion]

        if len(copper_atoms) > 1:
            if any(calculate_distance(atom1, atom2) <= distance_threshold for atom1 in copper_atoms for atom2 in copper_atoms if atom1 != atom2):
                homo_multinuclear_results.append(pdb_id)
            else:
                other_metals_near_copper = any(
                    any(calculate_distance(copper_atom, atom) <= distance_threshold for atom in res if atom.element in other_metals)
                    for copper_atom in copper_atoms for model in structure for chain in model for res in chain
                )
                (multi_hetero_results if other_metals_near_copper else multi_mono_results).append(pdb_id)

    save_results_multinuclear(multi_hetero_results, multi_mono_results, homo_multinuclear_results, directory)

# Function to save results of categorized PDB files
def save_results(mono_hetero_results, mono_results, directory):
    for results, subdir in [(mono_hetero_results, 'mono_hetero2'), (mono_results, 'mono2')]:
        output_dir = os.path.join(directory, subdir)
        os.makedirs(output_dir, exist_ok=True)
        pd.DataFrame(results, columns=['PDB ID']).to_excel(os.path.join(output_dir, f'{subdir}.xlsx'), index=False)
        for pdb_id in results:
            shutil.copy(os.path.join(directory, f"{pdb_id}.pdb"), output_dir)
        print(f"Results saved to '{output_dir}/{subdir}.xlsx' and corresponding PDB files copied.")

# Function to save results of categorized multinuclear PDB files
def save_results_multinuclear(multi_hetero_results, multi_mono_results, homo_multinuclear_results, directory):
    for results, subdir in [(multi_hetero_results, 'multi_hetero'), (multi_mono_results, 'multi_mono'), (homo_multinuclear_results, 'homo_multinuclear')]:
        output_dir = os.path.join(directory, subdir)
        os.makedirs(output_dir, exist_ok=True)
        pd.DataFrame(results, columns=['PDB ID']).to_excel(os.path.join(output_dir, f'{subdir}.xlsx'), index=False)
        for pdb_id in results:
            shutil.copy(os.path.join(directory, f"{pdb_id}.pdb"), output_dir)
        print(f"Results saved to '{output_dir}/{subdir}.xlsx' and corresponding PDB files copied.")

# Function to copy directories and files
def copy_directories_and_files(src_dirs, dst_dir):
    os.makedirs(dst_dir, exist_ok=True)
    for src_dir in src_dirs:
        if os.path.exists(src_dir):
            dst_subdir = os.path.join(dst_dir, os.path.basename(src_dir))
            shutil.copytree(src_dir, dst_subdir)
            for root, _, files in os.walk(src_dir):
                for file in files:
                    if file.endswith('.pdb'):
                        shutil.copy(os.path.join(root, file), dst_dir)
            print(f"Directory '{src_dir}' and PDB files copied to '{dst_subdir}'.")

# Main script
# 1. Download PDB files
excel_file = "D://240707_copper/cu_count_results0.xlsx"
df = pd.read_excel(excel_file)
pdb_codes = df["PDB ID"].tolist()
pdb_directory = "D://240707_copper/"
os.makedirs(pdb_directory, exist_ok=True)

failed_pdb_ids = [pdb_code for pdb_code in pdb_codes if not download_pdb(pdb_code, pdb_directory)]
pd.DataFrame({"PDB ID": failed_pdb_ids}).to_excel("D://240707_copper/Cu_failed.xlsx", index=False)
print(f"Failed PDB IDs saved to D://240707_copper/Cu_failed.xlsx")

# 2. Save categorized metal atoms to Excel and copy corresponding PDB files
save_metal_atoms_to_excel(pdb_directory, 'D://240707_copper/CU_combined')

# 3. Count 'CU' atoms based on the PDB IDs in cu_atoms.xlsx and categorize PDB files
count_and_categorize_cu_atoms(os.path.join('D://240707_copper/CU_combined', 'cu_atoms.xlsx'), 'D://240707_copper/CU_combined', os.path.join('D://240707_copper/CU_combined', 'cu_count_results.xlsx'))

# 4. Categorize and copy PDB files based on the presence of copper and other metals
categorize_pdb_files('D://240707_copper/CU_combined/metal_count_1/')

# 5. Categorize and copy PDB files based on the presence of multiple copper and other metals
categorize_pdb_files_multinuclear('D://240707_copper/CU_combined/metal_count_greater_than_1/')

# 6. Copy 'mono_2' and 'multi_mono' directories and individual PDB files to the new directory
copy_directories_and_files(['D:/240707_copper/CU_combined/metal_count_1/mono2', 'D:/240707_copper/CU_combined/metal_count_greater_than_1/multi_mono'], 'D:/240707_copper/CU_combined/mono')


In [ ]:
#split 2(final): Remove sequence redundancy
# Function to calculate coordination information for copper atoms
def calculate_coordination_info(structure, cu_distance_thresholds, allowed_residues, pdb_id):
    coordination_info = {}

    for model in structure:
        for chain in model:
            for res in chain:
                if res.get_resname() == 'CU':
                    cu_atom = res['CU']
                    coord_info = set()
                    for model2 in structure:
                        for chain2 in model2:
                            for res2 in chain2:
                                if res2.get_resname() in allowed_residues:
                                    for atom in res2:
                                        dist = calculate_distance(cu_atom, atom)
                                        key = f"{res2.get_resname()}_{atom.id}"
                                        if key in cu_distance_thresholds and cu_distance_thresholds[key][0] <= dist <= cu_distance_thresholds[key][1]:
                                            coord_info.add((pdb_id, structure.header['idcode'], chain.id, res.id[1], res.resname, chain2.id, res2.id[1], res2.resname, atom.id))

                    coord_count = len(coord_info)
                    if coord_count in coordination_info:
                        coordination_info[coord_count].extend(coord_info)
                    else:
                        coordination_info[coord_count] = list(coord_info)

    return coordination_info

# Function to parse PDB files in a directory
def parse_pdb_files(directory_path):
    pdb_parser = PDBParser(QUIET=True)
    return [(f[:4], pdb_parser.get_structure(f[:4], os.path.join(directory_path, f))) for f in os.listdir(directory_path) if f.endswith(".pdb")]

# Function to create a DataFrame from coordination information
def create_dataframe(info_list):
    base_cols = ['PDB ID', 'Entry ID', 'Metal Chain ID', 'Metal Residue number', 'Metal', 'Chain ID', 'Residue number', 'Residue name']
    dynamic_cols = [f'Binding atom {i+1}' for i in range(max(len(item) - len(base_cols) for item in info_list))]
    return pd.DataFrame(info_list, columns=base_cols + dynamic_cols).fillna(np.nan)

# Function to save DataFrame to an Excel file
def save_to_excel(df, output_path):
    df.to_excel(output_path, index=False)

# Function to extract sequences from PDB files
def extract_sequences(input_file, pdb_directory, output_file):
    input_df = pd.read_excel(input_file)
    sequences_data = []

    for index, row in input_df.iterrows():
        pdb_path = os.path.join(pdb_directory, f"{row['PDB ID']}.pdb")
        try:
            sequences = [str(record.seq) for record in SeqIO.parse(pdb_path, "pdb-atom") if record.annotations.get("chain") == row['Chain ID']]
            sequences_data.extend([{"PDB ID": row['PDB ID'], "Chain ID": row['Chain ID'], "Sequence": seq} for seq in sequences] or [{"PDB ID": row['PDB ID'], "Chain ID": row['Chain ID'], "Sequence": "No sequence found"}])
        except Exception as e:
            print(f"Error processing {pdb_path}: {e}")

    pd.DataFrame(sequences_data).to_excel(output_file, index=False)
    print("Sequences saved to", output_file)

# Function to align sequences and identify unique ones
def align_sequences(input_file, threshold, output_file_unique, output_file_excluded):
    df = pd.read_excel(input_file)
    unique_sequences, excluded_sequences = [], []

    start_time = time.time()

    for i, row in df.iterrows():
        sequence, pdb_id = row["Sequence"], row["PDB ID"]
        is_unique = True

        for unique_pdb_id, unique_sequence in unique_sequences:
            alignment = pairwise2.align.globalxx(sequence, unique_sequence, one_alignment_only=True)
            if sum(a == b for a, b in zip(alignment[0][0], alignment[0][1])) / len(alignment[0][0]) * 100 >= threshold:
                excluded_sequences.append((pdb_id, sequence))
                print(f"PDB ID {pdb_id} excluded due to high identity with {unique_pdb_id}.")
                is_unique = False
                break

        if is_unique:
            unique_sequences.append((pdb_id, sequence))
            print(f"PDB ID {pdb_id} is unique.")

        print(f"Progress: {i+1}/{len(df)} | Elapsed Time: {time.time() - start_time:.2f}s")

    pd.DataFrame(unique_sequences, columns=["PDB ID", "Sequence"]).to_excel(output_file_unique, index=False)
    pd.DataFrame(excluded_sequences, columns=["PDB ID", "Sequence"]).to_excel(output_file_excluded, index=False)
    print("Sequence identity analysis completed.")

# Function to copy PDB files for unique sequences
def copy_pdb_files(df, source_dir, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)
    for pdb_id in df['PDB ID'].tolist():
        src_file = os.path.join(source_dir, f"{pdb_id}.pdb")
        if os.path.exists(src_file):
            shutil.copy(src_file, destination_dir)
            print(f"Copied: {pdb_id}.pdb to {destination_dir}")
        else:
            print(f"File not found: {pdb_id}.pdb")

# Main script continuation
# Define allowed residues for coordination calculation
allowed_residues = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'LYS', 'MET', 'PHE', 'SER', 'THR', 'TYR', 'VAL'}

# Define distance thresholds for copper coordination with various residues
cu_distance_thresholds = {
    'HIS_ND1': (0, 2.7), 'HIS_NE2': (0, 2.7), 'HIS_N': (0, 2.7), 'HIS_O': (0, 2.95), 'GLU_OE': (0, 2.95), 'ASP_OD': (0, 2.95),
    'ASP_N': (0, 2.7), 'ASP_O': (0, 2.95), 'GLU_N': (0, 2.7), 'GLU_O': (0, 2.95), 'ALA_N': (0, 2.7), 'ALA_O': (0, 2.95),
    'CYS': (0, 2.75), 'MET': (0, 2.75), 'ARG_NH1': (0, 2.7), 'ARG_NH2': (0, 2.7), 'ARG_N': (0, 2.7), 'ARG_O': (0, 2.95),
    'ARG_NE': (0, 2.7), 'ASN_OD1': (0, 2.95), 'ASN_ND2': (0, 2.7), 'ASN_N': (0, 2.7), 'ASN_O': (0, 2.95), 'GLN_OE1': (0, 2.95),
    'GLN_NE2': (0, 2.7), 'GLN_O': (0, 2.95), 'GLN_N': (0, 2.7), 'GLY_N': (0, 2.7), 'GLY_O': (0, 2.95), 'LYS_NZ': (0, 2.7),
    'LYS_N': (0, 2.7), 'LYS_O': (0, 2.95), 'SER_OG': (0, 2.95), 'SER_N': (0, 2.7), 'SER_O': (0, 2.95), 'THR_OG1': (0, 2.95),
    'THR_N': (0, 2.7), 'THR_O': (0, 2.95), 'TYR_OH': (0, 2.95), 'TYR_N': (0, 2.7), 'TYR_O': (0, 2.95)
}

# Parse PDB files
structures = parse_pdb_files('D:/240707_copper/CU_combined/mono')

# Calculate coordination info
info_list = [info for pdb_id, structure in structures for info in sum(calculate_coordination_info(structure, cu_distance_thresholds, allowed_residues, pdb_id).values(), [])]

# Create DataFrame and save to Excel
df_coordination = create_dataframe(info_list)
save_to_excel(df_coordination, 'D:/240707_copper/CU_combined/mono/mono_coordination.xlsx')

print(f"Excel file saved to: D:/240707_copper/CU_combined/mono/mono_coordination.xlsx")

# Extract sequences from PDB files and save to Excel
extract_sequences('D:/240707_copper/CU_combined/mono/mono_coordination.xlsx', 'D:/240707_copper/CU_combined/mono/', 'D:/240707_copper/CU_combined/mono/mono_coordination_sequence.xlsx')

# Align sequences and identify unique ones
align_sequences('D:/240707_copper/CU_combined/mono/mono_coordination_sequence.xlsx', 50.0, 'D:/240707_copper/CU_combined/mono/mono_coordination_sequence_unique.xlsx', 'D:/240707_copper/CU_combined/mono/mono_coordination_sequence_exclude.xlsx')

# Copy PDB files for unique sequences
df_unique = pd.read_excel('D:/240707_copper/CU_combined/mono/mono_coordination_sequence_unique.xlsx')
copy_pdb_files(df_unique, 'D:/240707_copper/CU_combined/mono/', 'D:/240707_copper/CU_combined/mono/final')

print("File copying completed.")

In [ ]:
#split 3(final): Filter out coordination # 3 with distance parameter from metalPDB
import os
import pandas as pd
from Bio import PDB
import numpy as np

# Define the set of allowed residues
allowed_residues = {'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'LYS', 'MET', 'PHE', 'SER', 'THR', 'TYR', 'VAL'}

def calculate_coordination_info(structure, cu_distance_thresholds, allowed_residues, pdb_id):
    coordination_info = {}
    distances_info = []

    def calculate_distance(atom1, atom2):
        # Calculate the Euclidean distance between two atoms
        x1, y1, z1 = atom1.get_coord()
        x2, y2, z2 = atom2.get_coord()
        distance = ((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)**0.5
        return distance

    for cu_model_id, cu_model in enumerate(structure):
        for cu_chain_id, cu_chain in enumerate(cu_model):
            for cu_residue in cu_chain:
                if cu_residue.get_resname() == 'CU':
                    cu_atom = cu_residue['CU']
                    coordination_info_for_cu = set()
                    processed_residues = set()

                    for model_id, model in enumerate(structure):
                        for chain_id, chain in enumerate(model):
                            for residue in chain:
                                if residue.get_resname() not in allowed_residues:
                                    continue

                                coord_atoms = set()  # Use a set to store unique coordinated atoms for the current residue

                                for atom in residue:
                                    atom_name = atom.id

                                    if residue.get_resname() == 'HIS' and atom_name == 'ND1':
                                        min_distance, max_distance = cu_distance_thresholds['HIS_ND1']
                                    elif residue.get_resname() == 'HIS' and atom_name == 'NE2':
                                        min_distance, max_distance = cu_distance_thresholds['HIS_NE2']
                                    elif residue.get_resname() == 'HIS' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['HIS_N']
                                    elif residue.get_resname() == 'HIS' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['HIS_O']
                                    elif residue.get_resname() == 'GLU' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['GLU_N']
                                    elif residue.get_resname() == 'GLU' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['GLU_O']
                                    elif residue.get_resname() == 'ASP' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['ASP_N']
                                    elif residue.get_resname() == 'ASP' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['ASP_O']
                                    elif residue.get_resname() == 'GLU' and atom_name == 'OE1':
                                        min_distance, max_distance = cu_distance_thresholds['GLU_OE']
                                    elif residue.get_resname() == 'GLU' and atom_name == 'OE2':
                                        min_distance, max_distance = cu_distance_thresholds['GLU_OE']
                                    elif residue.get_resname() == 'ASP' and atom_name == 'OD1':
                                        min_distance, max_distance = cu_distance_thresholds['ASP_OD']
                                    elif residue.get_resname() == 'ASP' and atom_name == 'OD2':
                                        min_distance, max_distance = cu_distance_thresholds['ASP_OD']
                                    elif residue.get_resname() == 'ALA' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['ALA_N']
                                    elif residue.get_resname() == 'ALA' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['ALA_O']
                                    elif residue.get_resname() == 'ARG' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['ARG_N']
                                    elif residue.get_resname() == 'ARG' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['ARG_O']
                                    elif residue.get_resname() == 'ASN' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['ASN_N']
                                    elif residue.get_resname() == 'ASN' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['ASN_O']
                                    elif residue.get_resname() == 'GLN' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['GLN_N']
                                    elif residue.get_resname() == 'GLN' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['GLN_O']
                                    elif residue.get_resname() == 'LYS' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['LYS_N']
                                    elif residue.get_resname() == 'LYS' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['LYS_O']
                                    elif residue.get_resname() == 'SER' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['SER_N']
                                    elif residue.get_resname() == 'SER' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['SER_O']
                                    elif residue.get_resname() == 'THR' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['THR_N']
                                    elif residue.get_resname() == 'THR' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['THR_O']
                                    elif residue.get_resname() == 'TYR' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['TYR_N']
                                    elif residue.get_resname() == 'TYR' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['TYR_O']
                                    elif residue.get_resname() == 'CYS' and atom_name == 'SG':
                                        min_distance, max_distance = cu_distance_thresholds['CYS']
                                    elif residue.get_resname() == 'MET' and atom_name == 'SD':
                                        min_distance, max_distance = cu_distance_thresholds['MET']
                                    elif residue.get_resname() == 'ARG' and atom_name == 'NH1':
                                        min_distance, max_distance = cu_distance_thresholds['ARG_NH1']
                                    elif residue.get_resname() == 'ARG' and atom_name == 'NH2':
                                        min_distance, max_distance = cu_distance_thresholds['ARG_NH2']
                                    elif residue.get_resname() == 'ARG' and atom_name == 'NE':
                                        min_distance, max_distance = cu_distance_thresholds['ARG_NE']
                                    elif residue.get_resname() == 'ASN' and atom_name == 'OD1':
                                        min_distance, max_distance = cu_distance_thresholds['ASN_OD1']
                                    elif residue.get_resname() == 'ASN' and atom_name == 'ND2':
                                        min_distance, max_distance = cu_distance_thresholds['ASN_ND2']
                                    elif residue.get_resname() == 'GLN' and atom_name == 'OE1':
                                        min_distance, max_distance = cu_distance_thresholds['GLN_OE1']
                                    elif residue.get_resname() == 'GLN' and atom_name == 'NE2':
                                        min_distance, max_distance = cu_distance_thresholds['GLN_NE2']
                                    elif residue.get_resname() == 'GLY' and atom_name == 'N':
                                        min_distance, max_distance = cu_distance_thresholds['GLY_N']
                                    elif residue.get_resname() == 'GLY' and atom_name == 'O':
                                        min_distance, max_distance = cu_distance_thresholds['GLY_O']
                                    elif residue.get_resname() == 'LYS' and atom_name == 'NZ':
                                        min_distance, max_distance = cu_distance_thresholds['LYS_NZ']
                                    elif residue.get_resname() == 'SER' and atom_name == 'OG':
                                        min_distance, max_distance = cu_distance_thresholds['SER_OG']
                                    elif residue.get_resname() == 'THR' and atom_name == 'OG1':
                                        min_distance, max_distance = cu_distance_thresholds['THR_OG1']
                                    elif residue.get_resname() == 'TYR' and atom_name == 'OH':
                                        min_distance, max_distance = cu_distance_thresholds['TYR_OH']
                                    else:
                                        min_distance, max_distance = 0, 0

                                    # Calculate the distance between cu_atom and atom
                                    distance = calculate_distance(cu_atom, atom)

                                    if min_distance <= distance <= max_distance:
                                        coord_atoms.add((atom_name, residue.get_id()[1], distance))
                                        distances_info.append((pdb_id, cu_chain.get_id(), cu_residue.get_id()[1], cu_residue.get_resname(),
                                                              chain.get_id(), residue.get_id()[1], residue.get_resname(), atom_name, distance))

                                if coord_atoms:
                                    sorted_coord_atoms = sorted(coord_atoms, key=lambda x: x[1])[:3]
                                    coord_residue_info = (
                                        pdb_id,
                                        structure.header['idcode'],
                                        cu_chain.get_id(),
                                        cu_residue.get_id()[1],
                                        cu_residue.get_resname(),
                                        chain.get_id(),
                                        residue.get_id()[1],
                                        residue.get_resname()
                                    )

                                    for coord_atom_info in sorted_coord_atoms:
                                        coord_residue_info += (coord_atom_info[0],)

                                    coordination_info_for_cu.add(coord_residue_info)
                                processed_residues.add(residue.get_id())

                    coordinated_residues_count = len(coordination_info_for_cu)

                    if coordinated_residues_count in coordination_info:
                        coordination_info[coordinated_residues_count].extend(coordination_info_for_cu)
                    else:
                        coordination_info[coordinated_residues_count] = list(coordination_info_for_cu)

    return coordination_info, distances_info


# Directory containing PDB files
pdb_directory = 'D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear'

# Dictionary defining distance thresholds for copper coordination with various residues
# Define distance thresholds for each coordination
cu_distance_thresholds = {
    'HIS_ND1': (0, 2.7),
    'HIS_NE2': (0, 2.7),
    'HIS_N': (0, 2.7),
    'HIS_O': (0, 2.95),
    'GLU_OE': (0, 2.95),
    'ASP_OD': (0, 2.95),
    'ASP_N': (0, 2.7),
    'ASP_O': (0, 2.95),
    'GLU_N': (0, 2.7),
    'GLU_O': (0, 2.95),
    'ALA_N': (0, 2.7),
    'ALA_O': (0, 2.95),
    'CYS': (0, 2.75),
    'MET': (0, 2.75),
    'ARG_NH1': (0, 2.7),
    'ARG_NH2': (0, 2.7),
    'ARG_N': (0, 2.7),
    'ARG_O': (0, 2.95),
    'ARG_NE': (0, 2.7),
    'ASN_OD1': (0, 2.95),
    'ASN_ND2': (0, 2.7),
    'ASN_N': (0, 2.7),
    'ASN_O': (0, 2.95),
    'GLN_OE1': (0, 2.95),
    'GLN_NE2': (0, 2.7),
    'GLN_O': (0, 2.95),
    'GLN_N': (0, 2.7),
    'GLY_N': (0, 2.7),
    'GLY_O': (0, 2.95),
    'LYS_NZ': (0, 2.7),
    'LYS_N': (0, 2.7),
    'LYS_O': (0, 2.95),
    'SER_OG': (0, 2.95),
    'SER_N': (0, 2.7),
    'SER_O': (0, 2.95),
    'THR_OG1': (0, 2.95),
    'THR_N': (0, 2.7),
    'THR_O': (0, 2.95),
    'TYR_OH': (0, 2.95),
    'TYR_N': (0, 2.7),
    'TYR_O': (0, 2.95),
}

# Initialize a dictionary to store coordination information for different coordination numbers
coordination_info_dict = {}
distances_info_list = []

# Loop through each PDB file in the specified directory
for filename in os.listdir(pdb_directory):
    if filename.endswith('.pdb'):
        pdb_file_path = os.path.join(pdb_directory, filename)
        pdb_id = filename.split('.')[0]  # Extract the PDB ID from the filename

        # Parse the PDB file
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure('protein', pdb_file_path)

        # Calculate coordination information for the current PDB file
        coordination_info, distances_info = calculate_coordination_info(structure, cu_distance_thresholds, allowed_residues, pdb_id)

        # Get the number of columns from the first row of data (if available)
        num_columns = len(next(iter(coordination_info.values()), [])) if coordination_info else 0

        # Generate column names dynamically
        columns = [f'Column{i}' for i in range(num_columns)]

        # Add the coordination information to the coordination_info_dict
        for coordination_number, info_list in coordination_info.items():
            if coordination_number in coordination_info_dict:
                coordination_info_dict[coordination_number].extend(info_list)
            else:
                coordination_info_dict[coordination_number] = info_list

        # Add the distances information to the distances_info_list
        distances_info_list.extend(distances_info)

# Create and save Excel files for all coordination numbers
for coordination_number, info_list in coordination_info_dict.items():
    # Generate columns dynamically based on the maximum number of columns in the data
    max_columns = max(len(row) for row in info_list) if info_list else 0
    columns = ['Entry ID', 'PDB ID', 'Metal Chain ID', 'Metal Residue number', 'Metal', 'Chain ID', 'Residue number', 'Residue name']

    # Add binding atom columns based on max number of binding atoms found
    for i in range(1, max_columns - 8 + 1):
        columns.append(f'Binding atom{i}')

    # Create a DataFrame with a fixed number of columns and fill missing values with NaN
    df_coordination = pd.DataFrame(info_list, columns=columns).fillna(np.nan)

    output_excel_path = f'D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_{coordination_number}.xlsx'
    df_coordination.to_excel(output_excel_path, index=False)

print('Data saved to properly formatted Excel files for all coordination numbers.')

Data saved to properly formatted Excel files for all coordination numbers.


In [ ]:
#split (final): 4: change the format of excel file
import pandas as pd

# Load the Excel file
coordination_df = pd.read_excel('D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_3.xlsx')

# Remove extra single quotes from column names
coordination_df.columns = coordination_df.columns.str.replace("'", "")

# Prepare the data for horizontal arrangement based on PDB_ID, Metal Chain ID, and Metal Residue number
grouped = coordination_df.groupby(['Entry ID', 'Metal Chain ID', 'Metal Residue number'])

# Create a new DataFrame to hold the horizontally arranged data
horizontal_data = []

for name, group in grouped:
    row = list(name)
    for _, data in group.iterrows():
        row.extend([
            data['Chain ID'], data['Residue number'], data['Residue name'], data['Binding atom1']
        ])
    horizontal_data.append(row)

# Determine the column names for the new DataFrame
max_columns = max(len(row) for row in horizontal_data)
columns = ['Entry ID', 'Metal Chain ID', 'Metal Residue number'] + \
    [item for i in range((max_columns - 3) // 4) for item in [f'Chain ID{i+1}', f'Residue_number{i+1}', f'Residue_name{i+1}', f'Binding atom{i+1}']]

# Create the horizontally arranged DataFrame
horizontal_df = pd.DataFrame(horizontal_data, columns=columns)

# Save the resulting DataFrame to a new Excel file
horizontal_df.to_excel('D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_3_format.xlsx', index=False)

In [ ]:
#split (final): 4: calculate the theta aangle
import pandas as pd

# Load the Excel file
coordination_df = pd.read_excel('D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_3_format.xlsx')

# Remove extra single quotes from column names
coordination_df.columns = coordination_df.columns.str.replace("'", "")

# Prepare the data for horizontal arrangement based on PDB_ID, Metal Chain ID, and Metal Residue number
grouped = coordination_df.groupby(['Entry ID', 'Metal Chain ID', 'Metal Residue number'])

# Create a new DataFrame to hold the horizontally arranged data
horizontal_data = []

for name, group in grouped:
    row = list(name)
    for _, data in group.iterrows():
        row.extend([
            data['Chain ID'], data['Residue number'], data['Residue name'], data['Binding atom1']
        ])
    horizontal_data.append(row)

# Determine the column names for the new DataFrame
max_columns = max(len(row) for row in horizontal_data)
columns = ['Entry ID', 'Metal Chain ID', 'Metal Residue number'] + \
    [item for i in range((max_columns - 3) // 4) for item in [f'Chain ID{i+1}', f'Residue_number{i+1}', f'Residue_name{i+1}', f'Binding atom{i+1}']]

# Create the horizontally arranged DataFrame
horizontal_df = pd.DataFrame(horizontal_data, columns=columns)

# Save the resulting DataFrame to a new Excel file
horizontal_df.to_excel('D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_3_theta.xlsx', index=False)

In [ ]:
#split (final): 5: calculate the pie angle
import numpy as np
import pandas as pd
from math import acos, degrees

# Function to calculate the angle between two vectors
def calculate_angle(vector1, vector2):
    """
    Calculate the angle between two vectors.
    """
    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)
    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = degrees(acos(dot_product))
    return angle

# Read data from the Excel file
input_file_path = 'D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_3_theta.xlsx'
data = pd.read_excel(input_file_path)

# Process each row to calculate angles
def calculate_angles_for_row(row):
    try:
        # Vectors for residue pair 1 (1-2)
        vector_CA1_CA2 = np.array([row['Calpha_X_2'] - row['Calpha_X_1'],
                                   row['Calpha_Y_2'] - row['Calpha_Y_1'],
                                   row['Calpha_Z_2'] - row['Calpha_Z_1']])
        vector_CB1_CB2 = np.array([row['Cbeta_X_2'] - row['Cbeta_X_1'],
                                   row['Cbeta_Y_2'] - row['Cbeta_Y_1'],
                                   row['Cbeta_Z_2'] - row['Cbeta_Z_1']])
        angle1 = calculate_angle(vector_CA1_CA2, vector_CB1_CB2)

        # Vectors for residue pair 2 (1-3)
        vector_CA1_CA3 = np.array([row['Calpha_X_3'] - row['Calpha_X_1'],
                                   row['Calpha_Y_3'] - row['Calpha_Y_1'],
                                   row['Calpha_Z_3'] - row['Calpha_Z_1']])
        vector_CB1_CB3 = np.array([row['Cbeta_X_3'] - row['Cbeta_X_1'],
                                   row['Cbeta_Y_3'] - row['Cbeta_Y_1'],
                                   row['Cbeta_Z_3'] - row['Cbeta_Z_1']])
        angle2 = calculate_angle(vector_CA1_CA3, vector_CB1_CB3)

        # Vectors for residue pair 3 (2-3)
        vector_CA2_CA3 = np.array([row['Calpha_X_3'] - row['Calpha_X_2'],
                                   row['Calpha_Y_3'] - row['Calpha_Y_2'],
                                   row['Calpha_Z_3'] - row['Calpha_Z_2']])
        vector_CB2_CB3 = np.array([row['Cbeta_X_3'] - row['Cbeta_X_2'],
                                   row['Cbeta_Y_3'] - row['Cbeta_Y_2'],
                                   row['Cbeta_Z_3'] - row['Cbeta_Z_2']])
        angle3 = calculate_angle(vector_CA2_CA3, vector_CB2_CB3)

        return pd.Series([angle1, angle2, angle3])

    except Exception as e:
        print(f"Error processing row: {e}")
        return pd.Series([None, None, None])

# Apply the function to each row and store the results in new columns
pi_angle_columns = ['Pi_Angle_1', 'Pi_Angle_2', 'Pi_Angle_3']
data[pi_angle_columns] = data.apply(calculate_angles_for_row, axis=1)

# Save the results back to an Excel file
output_path = 'D:/240701_All_copper_final/CU_combined/metal_count_greater_than_1/homo_multinuclear/new_3_theta_pie.xlsx'
data.to_excel(output_path, index=False)

print("Angles calculated and saved to:", output_path)